In [1]:
import pandas as pd
import re
import googlemaps
import numpy as np
import time

In [3]:
df = pd.read_json('lojas_renner.json')

In [4]:
df

,UF,estabelecimento,endereco,numero
0,AC,Shopping Via Verde,ESTRADA DA FLORESTA,1707
1,AL,Shopping Iguatemi Maceió,AVENIDA GUSTAVO PAIVA,2990
2,AL,Shopping Parque Shopping Maceió,AVENIDA COMENDADOR GUSTAVO PAIVA,5945
3,AL,Shopping Pátio Maceió,AVENIDA MENINO MARCELO,3800
4,AL,Shopping Arapiraca Garden,RUA JOSE LEITE BEZERRA,S/N
...,...,...,...,...
380,SP,Loja Avenida Paulista - RENNER,AVENIDA PAULISTA,1106
381,SP,Shopping Tamboré,AVENIDA PIRACEMA,669
382,SP,Shopping Pátio Paulista - RENNER,RUA TREZE DE MAIO,1947
383,SP,Shopping Light,RUA CORONEL XAVIER DE TOLEDO,23


In [5]:
df = df.rename(columns = {'estabelecimento': 'Estabelecimento', 'endereco': 'Endereço', 'numero':'Número'})

In [6]:
df

,UF,Estabelecimento,Endereço,Número
0,AC,Shopping Via Verde,ESTRADA DA FLORESTA,1707
1,AL,Shopping Iguatemi Maceió,AVENIDA GUSTAVO PAIVA,2990
2,AL,Shopping Parque Shopping Maceió,AVENIDA COMENDADOR GUSTAVO PAIVA,5945
3,AL,Shopping Pátio Maceió,AVENIDA MENINO MARCELO,3800
4,AL,Shopping Arapiraca Garden,RUA JOSE LEITE BEZERRA,S/N
...,...,...,...,...
380,SP,Loja Avenida Paulista - RENNER,AVENIDA PAULISTA,1106
381,SP,Shopping Tamboré,AVENIDA PIRACEMA,669
382,SP,Shopping Pátio Paulista - RENNER,RUA TREZE DE MAIO,1947
383,SP,Shopping Light,RUA CORONEL XAVIER DE TOLEDO,23


In [7]:
def melhorando_endereco(x): # adiciona a palavra renner pra cada elemento de Estabelecimento, pra facilitar o API do google maps
    
    pattern = re.compile(r'[Rr][Ee][Nn][Nn][Ee][Rr]')
    
    if re.search(pattern,x):
        
        return x
    return x + ' Renner'

In [8]:
df['Estabelecimento + Renner'] = df['Estabelecimento'].apply(melhorando_endereco)

In [9]:
df['Estabelecimento + Renner'] = df['UF'] + ' - ' + df['Estabelecimento + Renner'] #adicionando a UF ao nome Estabelecimento + Renner

In [10]:
df['Endereço Completo'] = df['Endereço'] + ' - ' + df['Número'] 

In [ ]:
"""
Tanto a coluna Endereço completo quanto 'Estabelecimento + Renner' nos ajudam a ter melhor acurácia na obtenção 
da latitude e longitude, pois uma sozinha pode gerar valores irreais para a localização. Como veremos em (1) 
"""

In [19]:
df = df.drop(['Endereço','Número'], axis = 1)

In [21]:
gmaps = googlemaps.Client(key='your_api_key')

# Geocoding an address
geocode_result = gmaps.geocode(df['Estabelecimento + Renner'].iloc[15]) #retorna lista de JSON's 

In [22]:
geocode_result # returns an empty list, a funcao tratando_erros() irá solucionar.

[]

In [28]:
def tratando_erros(x):
    
    if len(x) == 0:
        return '[]'
    return x

In [25]:
df['objeto']=df['Estabelecimento + Renner'].apply(lambda x : gmaps.geocode(x, {'country': 'BR'}))

In [35]:
df['objeto'] = df['objeto'].apply(tratando_erros)

In [36]:
df[df['objeto'] == '[]']                                  """(1)"""

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto
95,MG,Shopping Metropolitan Garden,MG - Shopping Metropolitan Garden Renner,"RODOVIA BR 381 FERNAO DIAS - Km n 492, 601 ao ...",[]
161,RJ,Shopping Top Shopping,RJ - Shopping Top Shopping Renner,AVENIDA GOVERNADOR ROBERTO SILVEIRA - 540,[]
170,RJ,Shopping Norte,RJ - Shopping Norte Renner,AVENIDA DOM HELDER CAMARA - 5474,[]
270,SP,Shopping Franca,SP - Shopping Franca Renner,AVENIDA RIO NEGRO - 1100,[]
276,SP,SHOPPING CENTER D,SP - SHOPPING CENTER D Renner,Avenida Cruzeiro do Sul - 1.100,[]
333,SP,Shopping Center Plaza,SP - Shopping Center Plaza Renner,AVENIDA DR.FRANCISCO MESQUITA - 1000,[]
362,SP,Shopping Center Penha,SP - Shopping Center Penha Renner,RUA DR JOAO RIBEIRO - 304,[]


In [39]:
df.head(20)

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto
0,AC,Shopping Via Verde,AC - Shopping Via Verde Renner,ESTRADA DA FLORESTA - 1707,"[{'address_components': [{'long_name': '2320',..."
1,AL,Shopping Iguatemi Maceió,AL - Shopping Iguatemi Maceió Renner,AVENIDA GUSTAVO PAIVA - 2990,"[{'address_components': [{'long_name': '3800',..."
2,AL,Shopping Parque Shopping Maceió,AL - Shopping Parque Shopping Maceió Renner,AVENIDA COMENDADOR GUSTAVO PAIVA - 5945,[{'address_components': [{'long_name': 'Parque...
3,AL,Shopping Pátio Maceió,AL - Shopping Pátio Maceió Renner,AVENIDA MENINO MARCELO - 3800,"[{'address_components': [{'long_name': '3800',..."
4,AL,Shopping Arapiraca Garden,AL - Shopping Arapiraca Garden Renner,RUA JOSE LEITE BEZERRA - S/N,"[{'address_components': [{'long_name': '493', ..."
5,AM,Loja Centro de Manaus,AM - Loja Centro de Manaus Renner,AVENIDA EDUARDO RIBEIRO - 600,"[{'address_components': [{'long_name': '600', ..."
6,AM,Shopping Ponta Negra,AM - Shopping Ponta Negra Renner,AVENIDA CORONEL TEIXEIRA - 5705,"[{'address_components': [{'long_name': '5705',..."
7,AM,Shopping Amazonas,AM - Shopping Amazonas Renner,AVENIDA DJALMA BATISTA - 482,"[{'address_components': [{'long_name': '482', ..."
8,AM,Shopping Manauara,AM - Shopping Manauara Renner,AVENIDA MARIO YPIRANGA - 1300,[{'address_components': [{'long_name': 'Manaua...
9,AM,Shopping Manaus Via Norte,AM - Shopping Manaus Via Norte Renner,AVENIDA ARQUITETO JOSE HENRIQUE BENTO RODRIGUE...,[{'address_components': [{'long_name': 'Monte ...


In [42]:
df_com_erros = df.iloc[df[(df['objeto'] == '[]')].index]

In [43]:
df_com_erros

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto
95,MG,Shopping Metropolitan Garden,MG - Shopping Metropolitan Garden Renner,"RODOVIA BR 381 FERNAO DIAS - Km n 492, 601 ao ...",[]
161,RJ,Shopping Top Shopping,RJ - Shopping Top Shopping Renner,AVENIDA GOVERNADOR ROBERTO SILVEIRA - 540,[]
170,RJ,Shopping Norte,RJ - Shopping Norte Renner,AVENIDA DOM HELDER CAMARA - 5474,[]
270,SP,Shopping Franca,SP - Shopping Franca Renner,AVENIDA RIO NEGRO - 1100,[]
276,SP,SHOPPING CENTER D,SP - SHOPPING CENTER D Renner,Avenida Cruzeiro do Sul - 1.100,[]
333,SP,Shopping Center Plaza,SP - Shopping Center Plaza Renner,AVENIDA DR.FRANCISCO MESQUITA - 1000,[]
362,SP,Shopping Center Penha,SP - Shopping Center Penha Renner,RUA DR JOAO RIBEIRO - 304,[]


In [44]:
df_com_erros['objeto'] = df_com_erros['Endereço Completo'].apply(lambda x: gmaps.geocode(x,{'country': 'BR'} ))

<ipython-input-44-0c0d5037fcb6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_com_erros['objeto'] = df_com_erros['Endereço Completo'].apply(lambda x: gmaps.geocode(x,{'country': 'BR'} ))


In [45]:
df_com_erros

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto
95,MG,Shopping Metropolitan Garden,MG - Shopping Metropolitan Garden Renner,"RODOVIA BR 381 FERNAO DIAS - Km n 492, 601 ao ...",[{'address_components': [{'long_name': '492-60...
161,RJ,Shopping Top Shopping,RJ - Shopping Top Shopping Renner,AVENIDA GOVERNADOR ROBERTO SILVEIRA - 540,"[{'address_components': [{'long_name': '540', ..."
170,RJ,Shopping Norte,RJ - Shopping Norte Renner,AVENIDA DOM HELDER CAMARA - 5474,"[{'address_components': [{'long_name': '5474',..."
270,SP,Shopping Franca,SP - Shopping Franca Renner,AVENIDA RIO NEGRO - 1100,"[{'address_components': [{'long_name': '1100',..."
276,SP,SHOPPING CENTER D,SP - SHOPPING CENTER D Renner,Avenida Cruzeiro do Sul - 1.100,[{'address_components': [{'long_name': 'Estrad...
333,SP,Shopping Center Plaza,SP - Shopping Center Plaza Renner,AVENIDA DR.FRANCISCO MESQUITA - 1000,"[{'address_components': [{'long_name': '1000',..."
362,SP,Shopping Center Penha,SP - Shopping Center Penha Renner,RUA DR JOAO RIBEIRO - 304,"[{'address_components': [{'long_name': '304', ..."


In [50]:
df_com_certos = df.iloc[df[~(df['objeto'] == '[]')].index]

In [53]:
df_total = pd.concat([df_com_certos,df_com_erros], axis = 0,join='outer')

In [55]:
len(df_total)

385

In [ ]:
df6.isnull().any()

In [56]:
df_total = df_total.sort_index()

In [57]:
df_total['latitude'] = df_total['objeto'].apply(lambda x: x[0]['geometry']['location']['lat'] if len(x) > 0 else [])

In [58]:
df_total['longitude'] = df_total['objeto'].apply(lambda x: x[0]['geometry']['location']['lng'] if len(x) > 0 else [])

In [59]:
df_total

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto,latitude,longitude
0,AC,Shopping Via Verde,AC - Shopping Via Verde Renner,ESTRADA DA FLORESTA - 1707,"[{'address_components': [{'long_name': '2320',...",-9.986120,-67.847468
1,AL,Shopping Iguatemi Maceió,AL - Shopping Iguatemi Maceió Renner,AVENIDA GUSTAVO PAIVA - 2990,"[{'address_components': [{'long_name': '3800',...",-9.555659,-35.758193
2,AL,Shopping Parque Shopping Maceió,AL - Shopping Parque Shopping Maceió Renner,AVENIDA COMENDADOR GUSTAVO PAIVA - 5945,[{'address_components': [{'long_name': 'Parque...,-9.627677,-35.699210
3,AL,Shopping Pátio Maceió,AL - Shopping Pátio Maceió Renner,AVENIDA MENINO MARCELO - 3800,"[{'address_components': [{'long_name': '3800',...",-9.558686,-35.746318
4,AL,Shopping Arapiraca Garden,AL - Shopping Arapiraca Garden Renner,RUA JOSE LEITE BEZERRA - S/N,"[{'address_components': [{'long_name': '493', ...",-9.734552,-36.655668
...,...,...,...,...,...,...,...
380,SP,Loja Avenida Paulista - RENNER,SP - Loja Avenida Paulista - RENNER,AVENIDA PAULISTA - 1106,"[{'address_components': [{'long_name': '1106',...",-23.564304,-46.652537
381,SP,Shopping Tamboré,SP - Shopping Tamboré Renner,AVENIDA PIRACEMA - 669,"[{'address_components': [{'long_name': '669', ...",-23.504342,-46.834366
382,SP,Shopping Pátio Paulista - RENNER,SP - Shopping Pátio Paulista - RENNER,RUA TREZE DE MAIO - 1947,[{'address_components': [{'long_name': 'Shoppi...,-23.570553,-46.643525
383,SP,Shopping Light,SP - Shopping Light Renner,RUA CORONEL XAVIER DE TOLEDO - 23,[{'address_components': [{'long_name': 'Shoppi...,-23.546577,-46.638632


In [ ]:
#vendo se há casos em que a latitude >>> 0

In [64]:
df_total[df_total['latitude']>0] 

,UF,Estabelecimento,Estabelecimento + Renner,Endereço Completo,objeto,latitude,longitude
201,RR,Shopping Roraima Garden,RR - Shopping Roraima Garden Renner,AVENIDA VILLE ROY - 1544,[{'address_components': [{'long_name': 'Roraim...,2.855355,-60.645626
202,RR,Shopping Pátio Roraima,RR - Shopping Pátio Roraima Renner,AVENIDA JOAO ALENCAR - 2181,"[{'address_components': [{'long_name': '2181',...",2.850390,-60.706551


In [67]:
df_total.to_excel('lojas-renner.xlsx')

In [66]:
import folium


map1 = folium.Map(
    location=[-15.77972, -47.92972],
    tiles='cartodbpositron',
    zoom_start=8,
)
df_total.apply(lambda row:folium.Marker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1


